# 1. Importing and preprocessing

In [2]:
# IMPORTS
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation, LeakyReLU
from keras.layers.noise import AlphaDropout
from keras.utils.generic_utils import get_custom_objects
from keras import backend as K
from keras.optimizers import Adam

# LOAD DATA
train_data = np.load('add_02.npz')
x_train = train_data['x']
y_train = train_data['y']

#print(x_train)
#print(y_train)

Using TensorFlow backend.


# 2. Readying functions and hyperparameters

In [5]:
# Readying neural network model
def build_cnn(activation, dropout_rate, optimizer):
    model = Sequential()
    #model.add(Dropout(0.2))
    model.add(Dense(32, input_shape=(3,))) 
    model.add(Activation(activation))
    #model.add(Dropout(0.1))
    model.add(Dense(1, activation='softmax'))
    
    model.compile(
        loss='binary_crossentropy', #损失函数（loss function），模型要将其最小化，可以通过字符串标识符/目标函数指定
        optimizer=optimizer, #优化器（opyimizer）,可以是优化器的字符串标识符，也可以是Optimizer类的实例
        metrics=['accuracy'] #指标（metricts）列表， 对于任何分类问题，需要将其设置为metrics = [‘accuracy’]
    )
    
    return model

# Add the GELU function to Keras
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})

#act_func = ['sigmoid', 'relu', 'elu', 'leaky-relu', 'selu', 'gelu']
act_func = ['sigmoid']#扩展型指数线性单元激活函数（SELU）高斯误差线性单元激活函数（GELU）

# 3. Fitting the data with multiple activation functions

In [7]:
result = []

for activation in act_func:
    print('\nTraining with -->{0}<-- activation function\n'.format(activation))
    
    model = build_cnn(activation=activation,
                      dropout_rate=0.2,
                      optimizer=Adam(clipvalue=0.5))
    '''
    model.fit(x=None, y=None, 
        batch_size=None, 
        epochs=1, 
        verbose=1, 
        callbacks=None, 
        validation_split=0.0, validation_data=None, 
        shuffle=True, 
        class_weight=None, sample_weight=None, 
        initial_epoch=0, steps_per_epoch=None, validation_steps=None, validation_freq=1)
    '''
    history = model.fit(x_train, y_train, 
                        batch_size=None, # 128 is faster, but less accurate. 16/32 recommended
                        epochs=100,
                        verbose=1,
                        validation_split=0.0,
                        validation_data=None ) 
    
    result.append(history)
    
    K.clear_session()
    del model

print(result)


Training with -->sigmoid<-- activation function

Epoch 1/100
7/7 [==============================] - 0s 25ms/step - loss: 6.5714 - accuracy: 0.5714
Epoch 2/100
7/7 [==============================] - 0s 285us/step - loss: 6.5714 - accuracy: 0.5714
Epoch 3/100
7/7 [==============================] - 0s 570us/step - loss: 6.5714 - accuracy: 0.5714
Epoch 4/100
7/7 [==============================] - 0s 285us/step - loss: 6.5714 - accuracy: 0.5714
Epoch 5/100
7/7 [==============================] - 0s 432us/step - loss: 6.5714 - accuracy: 0.5714
Epoch 6/100
7/7 [==============================] - 0s 570us/step - loss: 6.5714 - accuracy: 0.5714
Epoch 7/100
7/7 [==============================] - 0s 570us/step - loss: 6.5714 - accuracy: 0.5714
Epoch 8/100
7/7 [==============================] - 0s 145us/step - loss: 6.5714 - accuracy: 0.5714
Epoch 9/100
7/7 [==============================] - 0s 142us/step - loss: 6.5714 - accuracy: 0.5714
Epoch 10/100
7/7 [==============================] - 0s 285us

Epoch 83/100
7/7 [==============================] - 0s 285us/step - loss: 6.5714 - accuracy: 0.5714
Epoch 84/100
7/7 [==============================] - 0s 285us/step - loss: 6.5714 - accuracy: 0.5714
Epoch 85/100
7/7 [==============================] - 0s 570us/step - loss: 6.5714 - accuracy: 0.5714
Epoch 86/100
7/7 [==============================] - 0s 285us/step - loss: 6.5714 - accuracy: 0.5714
Epoch 87/100
7/7 [==============================] - 0s 285us/step - loss: 6.5714 - accuracy: 0.5714
Epoch 88/100
7/7 [==============================] - 0s 285us/step - loss: 6.5714 - accuracy: 0.5714
Epoch 89/100
7/7 [==============================] - 0s 288us/step - loss: 6.5714 - accuracy: 0.5714
Epoch 90/100
7/7 [==============================] - 0s 285us/step - loss: 6.5714 - accuracy: 0.5714
Epoch 91/100
7/7 [==============================] - 0s 285us/step - loss: 6.5714 - accuracy: 0.5714
Epoch 92/100
7/7 [==============================] - 0s 427us/step - loss: 6.5714 - accuracy: 0.5714


# 4. Graph the results

In [ ]:
def plot_act_func_results(results, activation_functions = []):
    plt.figure(figsize=(10,10))
    plt.style.use('dark_background')
    
    # Plot validation accuracy values
    for act_func in results:
        plt.plot(act_func.history['val_acc'])
        
    plt.title('Model accuracy')
    plt.ylabel('Test Accuracy')
    plt.xlabel('Epoch')
    plt.legend(activation_functions)
    plt.show()

    # Plot validation loss values
    plt.figure(figsize=(10,10))
    
    for act_func in results:
        plt.plot(act_func.history['val_loss'])
        
    plt.title('Model loss')
    plt.ylabel('Test Loss')
    plt.xlabel('Epoch')
    plt.legend(activation_functions)
    plt.show()

plot_act_func_results(result, act_func)

In [ ]:
new_act_arr = act_func[1:]
new_results = result[1:]

def plot_act_func_results(results, activation_functions = []):
    plt.figure(figsize=(10,10))
    plt.style.use('dark_background')
    
    # Plot validation accuracy values
    for act_func in results:
        plt.plot(act_func.history['val_acc'])
        
    plt.title('Model accuracy')
    plt.ylabel('Test Accuracy')
    plt.xlabel('Epoch')
    plt.legend(activation_functions)
    plt.show()

    # Plot validation loss values
    plt.figure(figsize=(10,10))
    
    for act_func in results:
        plt.plot(act_func.history['val_loss'])
        
    plt.title('Model loss')
    plt.ylabel('Test Loss')
    plt.xlabel('Epoch')
    plt.legend(activation_functions)
    plt.show()

plot_act_func_results(new_results, new_act_arr)

In [1]:
import numpy as np

# the training inputs 
# the last column is used to add non linearity to the clasification task
x = np.array([[0, 0, 1],
              [0, 1, 1],
              [1, 0, 1],
              [0, 1, 0],
              [1, 0, 0],
              [1, 1, 1],
              [0, 0, 0]])
    
# the training outputs
y = np.array([[0, 1, 1, 1, 1, 0, 0]]).T

np.savez('add_date.npz', x=x, y=y)
train_data = np.load('add_date.npz')
print(train_data['x'])
print(train_data['y'])

# the test inputs 
# the last column is used to add non linearity to the clasification task
x = np.array([[1, 0, 0]])
    
# the training outputs
y = np.array([[1]]).T

np.savez('add_date_t.npz', x=x, y=y)
test_data = np.load('add_date_t.npz')
print(test_data['x'])
print(test_data['y'])

[[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
[[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
[[1 0 0]]
[[1]]
